## Importing Modules

In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append('/home/junhokang/script')
import scjp

In [ ]:
# useful imports
import numpy as np
import scipy as scipy
import scanpy as sc
import scanpy.external as sce
import pandas as pd
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', None)
import pickle as pkl
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10,8]
import seaborn as sns
from collections import defaultdict, Counter
import networkx as nx
import igraph, re, glob
from bbknn import bbknn
from geosketch import gs
import scrublet as scr
import joblib as jl
from datetime import datetime
def timestamp():
    return datetime.now().strftime("%y%m%d%H%M")
import logging
import scipy.stats
import diffxpy.api as de
from SCCAF import *
import gseapy

In [ ]:
# setting scanpy
%matplotlib inline
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100, color_map='OrRd')
sc.logging.print_version_and_date()

In [ ]:
nb_name = 'MOG15.v04.ms_GSE138266_Annotate_Submit_220121'
version = '.'.join(nb_name.split('.')[:2])+'.'
data = '_'.join(nb_name.split('.')[-1:][0].split('_')[:-2])
print('Version:', version)
print('Data:', data)

In [ ]:
base_folder = '/home/junhokang/projects/02_mogad_new/'

## Loading Adata

In [ ]:
adata = sc.read('/home/junhokang/projects/02_mogad_new/99_script/write/MOG10.v01.ms_GSE138266_sccaf2.h5ad')

In [ ]:
adata.obs

In [ ]:
adata.obs = adata.obs[remain]

## QC check

In [ ]:
sc.pl.scatter(adata, x='n_counts', y='mito')
sc.pl.scatter(adata, x='n_counts', y='n_genes')

In [ ]:
scjp.us(adata,'anno_sccaf')

In [ ]:
scjp.us(adata,'Sample')

## re-Annotate 10X 3' dataset using SCCAF

In [ ]:
sc.tl.leiden(adata,resolution=0.3)

In [ ]:
scjp.us(adata,'leiden',legend_loc='on data',legend_fontsize=6)

In [ ]:
scjp.us(adata,'leiden',groups='7')

In [ ]:
scjp.us(adata,'anno_sccaf',legend_loc='on data',legend_fontsize=6)

In [ ]:
scjp.us(adata,'mito,n_genes')

In [ ]:
scjp.us(adata,'CD4,CD8B,NKG7,KLRB1,IL18R1,FOXP3,CCL5,MS4A1,LYZ,CD74,S100A8')

In [ ]:
# T_pan
scjp.us(adata,'CD3D,CD3E,LCK,TRAC')

In [ ]:
#NK_pan
scjp.us(adata,'GNLY,NKG7,NCAM1,KLRD1,NCR1,NCAM1')

In [ ]:
#NK1_CD16
scjp.us(adata,'FCGR3A,PRF1')

In [ ]:
#NK2_XCL1
scjp.us(adata,'SELL,XCL1')

In [ ]:
#B_pan
scjp.us(adata,'MS4A1,CD19,CD74,CD79A')

In [ ]:
#B_naive
scjp.us(adata,'CD37,IGHD')

In [ ]:
#B_memory
scjp.us(adata,'CD27,IGHM')

In [ ]:
#B_plasma
scjp.us(adata,'IGHG1,JCHAIN,CD38,TNFRSF17')

In [ ]:
#Myl_pan
scjp.us(adata,'LYZ')

In [ ]:
#Myl_Mast
scjp.us(adata,'KIT,TPSAB1,CPA3,FCGR2A,CD33,ENPP3')

In [ ]:
#Hp_Mgk
scjp.us(adata,'GNG11,CLU,ITGA2B')

In [ ]:
# Hp_Ery
scjp.us(adata,'GYPA,HBB')

## Subclustering

In [ ]:
scjp.us(adata,'anno_sccaf',legend_loc='on data',legend_fontsize=6)

In [ ]:
t = [x for x in sorted(set(adata.obs['anno_final'])) if x.startswith('T_')]
t.append('LQ_T')

In [ ]:
t

In [ ]:
tdata = scjp.get_subset(adata,adata.obs['anno_final'].isin(t))

In [ ]:
scjp.run_harmony(tdata,'PatientID')

In [ ]:
scjp.us(tdata,'anno_final')

In [ ]:
scjp.us(tdata,'anno_final',groups='LQ_T')

In [ ]:
sc.tl.leiden(tdata, resolution=2.0, key_added='L2_Round0')

In [ ]:
SCCAF_optimize_all(ad=tdata,min_acc=0.90, start='L2_Round0',prefix='L2',use='pca',basis='umap',c_iter=4)

In [ ]:
scjp.us(tdata,'anno_final',legend_loc='on data',legend_fontsize=6)

In [ ]:
scjp.us(tdata,'anno_sccaf',legend_loc='on data',legend_fontsize=6)

In [ ]:
scjp.us(tdata,'L2_Round3',legend_loc='on data',legend_fontsize=7)

In [ ]:
scjp.us(tdata,'mito,n_genes')

In [ ]:
scjp.us(tdata,'CD4,CD8B,NKG7,KLRB1,IL18R1,FOXP3,CCL5,MS4A1,LYZ,CD74,S100A8')

In [ ]:
#Hp_Mgk
scjp.us(tdata,'GNG11,CLU,ITGA2B')

In [ ]:
# Hp_Ery
scjp.us(tdata,'GYPA,HBB')

In [ ]:
#T_CD4 pan
scjp.us(tdata,'CD3D,CD3E,CD4,IL7R')

In [ ]:
#T_REG
scjp.us(tdata,'FOXP3,CTLA4')

In [ ]:
# CD4+ Naive T
scjp.us(tdata,'SELL,TCF7,CD4,CCR7,IL7R,FHIT,LEF1,MAL,NOSIP,LDHB,PIK3IP1')

In [ ]:
CD4NAIVE='SELL,TCF7,CD4,CCR7,IL7R,FHIT,LEF1,MAL,NOSIP,LDHB,PIK3IP1'.split(',')
sc.tl.score_genes(tdata,CD4NAIVE,score_name='CD4NAIVE')

In [ ]:
scjp.us(tdata,'CD4NAIVE')

In [ ]:
# CD4+ Effector Memory T
scjp.us(tdata,'IL7R,CCL5,FYB1,GZMK,IL32,GZMA,KLRB1,TRAC,LTB,AQP3')

In [ ]:
CD4TEM='IL7R,CCL5,FYB1,GZMK,IL32,GZMA,KLRB1,TRAC,LTB,AQP3'.split(',')
sc.tl.score_genes(tdata,CD4TEM,score_name='CD4TEM')

In [ ]:
scjp.us(tdata,'CD4TEM')

In [ ]:
# CD4 Proliferating
scjp.us(tdata,'MKI67,TOP2A,PCLAF,CENPF,TYMS,NUSAP1,ASPM,PTTG1,TPX2,RRM2')

In [ ]:
#T_CD4 Th17_RORC
scjp.us(tdata,'RORC,IL17A,KLRB1,IL23R,CCL20,CCR6')

In [ ]:
#T_CD4 Tfh
scjp.us(tdata,'CXCR5,CD200,CXCL13,BCL6,STAT3,MAF,PDCD1,CCR7,CXCR3,CCR6,CD40LG')

In [ ]:
#T_CD8 pan
scjp.us(tdata,'CD8A,GZMH')

In [ ]:
#T_CD8 naive
scjp.us(tdata,'CD8A,CD8B,CCR7')

In [ ]:
#T_CD8 memory
scjp.us(tdata,'CD3E,CD4,CD8B,CCL5')

In [ ]:
#T_GD
scjp.us(tdata,'TRDC,CD3D')

In [ ]:
#T_NK
scjp.us(tdata,'CD3D,CD3E,NKG7,KLRB1,ZNF683,CD8A,ZBTB16')

In [ ]:
#T_MAIT
scjp.us(tdata,'SLC4A10,KLRB1,IL18R1,CXCR6,CCR6,SATB1,TRAV1-2,ZBTB16,RORC,CCR7,TCF7')

In [ ]:
#NK_pan
scjp.us(tdata,'GNLY,NKG7,NCAM1,KLRD1,NCR1,NCAM1')

In [ ]:
#NK1_CD16
scjp.us(tdata,'FCGR3A,PRF1')

In [ ]:
#NK2_XCL1
scjp.us(tdata,'SELL,XCL1')

In [ ]:
# ILC
scjp.us(tdata,'KIT,IL1R1,TNFRSF4,TRDC,TTLL10,SOX4,TNFRSF18')

In [ ]:
ILC='KIT,IL1R1,TNFRSF4,TRDC,TTLL10,SOX4,TNFRSF18'.split(',')
sc.tl.score_genes(tdata,ILC,score_name='ILC')

In [ ]:
scjp.us(tdata,'ILC')

In [ ]:
sc.tl.leiden(tdata,resolution=0.3,restrict_to=('L2_Round3',['4']),key_added='sub')

In [ ]:
tdata.obs['sub'] = ['_'.join(x.split(','))for x in tdata.obs['sub']]

In [ ]:
scjp.us(tdata,'sub',legend_loc='on data',legend_fontsize=7)

In [ ]:
ct_anno = scjp.annotater(tdata,'anno_final',old_label='anno_final')

In [ ]:
m = scjp.markers.marker(tdata,'L2_Round3')
marker = m.plot_marker()

In [ ]:
ct_anno.update(tdata,'L2_Round3','0','T_CD4')
ct_anno.update(tdata,'L2_Round3','1','T_NK')
ct_anno.update(tdata,'L2_Round3','2','T_CD8_memory')
ct_anno.update(tdata,'L2_Round3','3','T_CD4')
ct_anno.update(tdata,'L2_Round3','4','T_CD4')
ct_anno.update(tdata,'L2_Round3','5','T_CD8_naive')
ct_anno.update(tdata,'L2_Round3','6','T_CD4')
ct_anno.update(tdata,'L2_Round3','7','T_CD4')
ct_anno.update(tdata,'L2_Round3','8','T_CD8_memory')
ct_anno.update(tdata,'L2_Round3','9','T_CD4')
ct_anno.update(tdata,'L2_Round3','10','T_CD4')
ct_anno.update(tdata,'L2_Round3','11','T_REG')
ct_anno.update(tdata,'L2_Round3','12','T_CD4')
ct_anno.update(tdata,'L2_Round3','13','T_MAIT')
ct_anno.update(tdata,'L2_Round3','14','T_CD8_memory')
ct_anno.update(tdata,'L2_Round3','15','LQ_Myl')
ct_anno.update(tdata,'L2_Round3','16','LQ_Myl')
ct_anno.update(tdata,'L2_Round3','17','LQ_B')

ct_anno.update(tdata,'sub','4_2','T_CD8_memory')

In [ ]:
scjp.us(tdata,'anno_final',legend_loc='on data',legend_fontsize=6)

In [ ]:
scjp.model.update_label(tdata,'anno_final',adata,'anno_final','anno_final',replace=True)

In [ ]:
scjp.us(adata,'anno_final',legend_loc='on data',legend_fontsize=6)

## Myeloid subsetting

In [ ]:
Myl = [x for x in sorted(set(adata.obs['anno_final'])) if x.startswith('Myl_')]
mdata = scjp.get_subset(adata,adata.obs['anno_final'].isin(Myl))

In [ ]:
sce.pp.harmony_integrate(mdata,'PatientID',adjusted_basis='X_pca')
scjp.sc_process(mdata,pid = 'ku')

In [ ]:
scjp.us(mdata,'anno_final', legend_loc='on data', legend_fontsize=7)

In [ ]:
sc.tl.leiden(mdata, resolution=2.0, key_added='L2_Round0')

In [ ]:
SCCAF_optimize_all(ad=mdata,min_acc=0.95, start='L2_Round0',prefix='L2',use='pca',basis='umap',c_iter=4)

In [ ]:
# mito, n_genes
scjp.us(mdata,'mito,n_genes')

In [ ]:
scjp.us(mdata,'L2_Round5',legend_loc='on data',legend_fontsize=6)

In [ ]:
scjp.us(mdata,'mito,n_genes,CDK1')

In [ ]:
scjp.us(mdata,'CD3D,CD4,CD8B,NKG7,KLRB1,IL18R1,FOXP3,CCL5,MS4A1,CD74,PI3,CXCL8,GNG11,LYZ')

In [ ]:
#DC_pan
scjp.us(mdata,'CD74,HLA-DRA')

In [ ]:
#cDC1
scjp.us(mdata,'CLEC9A,XCR1,ANPEP,FLT3,HLA-DPA1,CADM1,CAMK2D,IDO1,WDFY4,BATF3')

In [ ]:
#cDC2
scjp.us(mdata,'CD1C,FCER1A,HLA-DQA1,CLEC10A,SIRPA,HLA-DQA1')

In [ ]:
#cDC3_LAMP3
scjp.us(mdata,'LAMP3,CCR7,FSCN1,CD40')

In [ ]:
#DC_CD5
scjp.us(mdata,'AXL,SIGLEC6,ADAM33,SIGLEC1,CD22,CD5,PPP1R14A,DAB2')

In [ ]:
#ACY3_DC
scjp.us(mdata,'SYT2,ACY3,MACC1,GTF2IRD1,KIT,PIK3R6,LINC00299,TTN,PIGR,NUDT8')

In [ ]:
#pDC
scjp.us(mdata,'GZMB,IL3RA,CLEC4C,LILRA4,JCHAIN,TCF4,TNFRSF21,SERPINF1,ITM2C')

In [ ]:
#Mono/Mac_pan
scjp.us(mdata,'CD68,CD163,ITGAM')

In [ ]:
#Mono_CD14
scjp.us(mdata,'CD14,FCN1,S100A8,S100A9,CD163,EGR1')

In [ ]:
#Mono_CD16
scjp.us(mdata,'FCGR3A,LST1,LILRB2,C1QA,MAF,CSF1R')

In [ ]:
#Granulocyte,CD14_mono
scjp.us(mdata,'S100A8,S100A9')

In [ ]:
#Microglia, perivascular macrophages (LYVE1), CNS border-associated macrophages(STAB1 and CH25H)
scjp.us(mdata,'LYVE1,STAB1,CH25H,Tissue')

In [ ]:
#Microglia
scjp.us(mdata,'TMEM119,CX3CR1,TREM2,GPR34,P2RY12')

In [ ]:
#Activated Microglia
scjp.us(mdata,'CX3CR1,CD68,SIRPA,CD47,CD40,CD80,CD28')

In [ ]:
#Neutrophil
scjp.us(mdata,'PI3,CHI3L1,ITGAM,CXCL8,ANXA3,IFITM2')

In [ ]:
#Mast
scjp.us(mdata,'KIT,TPSAB1,CPA3,FCGR2A,CD33,CD63,ENPP3')

In [ ]:
sc.tl.leiden(mdata,resolution=0.5,restrict_to=('L2_Round5',['8']),key_added='sub')

In [ ]:
sc.tl.leiden(mdata,resolution=0.5,restrict_to=('sub',['10']),key_added='sub')

In [ ]:
sc.tl.leiden(mdata,resolution=0.3,restrict_to=('sub',['15']),key_added='sub')

In [ ]:
mdata.obs['sub'] = ['_'.join(x.split(','))for x in mdata.obs['sub']]

In [ ]:
scjp.us(mdata,'sub',legend_loc='on data',legend_fontsize=7)

In [ ]:
sc.tl.leiden(mdata,resolution=0.5)

In [ ]:
scjp.us(mdata,'leiden',legend_loc='on data',legend_fontsize=7)

In [ ]:
ct_anno = scjp.annotater(mdata,'anno_final',old_label='anno_final')

In [ ]:
ct_anno.update(mdata,'leiden','7','LQ_T')
ct_anno.update(mdata,'leiden','12','LQ_B')

In [ ]:
ct_anno.update(mdata,'L2_Round5','0','Myl_MonoCD14')
ct_anno.update(mdata,'L2_Round5','1','Myl_Microglia')
ct_anno.update(mdata,'L2_Round5','2','Myl_MonoCD14')
ct_anno.update(mdata,'L2_Round5','3','Myl_DC2')
ct_anno.update(mdata,'L2_Round5','4','Myl_MonoCD16')
ct_anno.update(mdata,'L2_Round5','5','LQ_T')
ct_anno.update(mdata,'L2_Round5','6','Myl_DC2')
ct_anno.update(mdata,'L2_Round5','7','Myl_pDC')
ct_anno.update(mdata,'L2_Round5','8','Myl_Microglia')
ct_anno.update(mdata,'L2_Round5','9','LQ_Doublet')
ct_anno.update(mdata,'L2_Round5','10','Myl_DC2')
ct_anno.update(mdata,'L2_Round5','11','Myl_Microglia')
ct_anno.update(mdata,'L2_Round5','12','LQ_T')
ct_anno.update(mdata,'L2_Round5','13','Myl_Microglia')
ct_anno.update(mdata,'L2_Round5','14','Myl_DC1')
ct_anno.update(mdata,'L2_Round5','16','LQ_B')
ct_anno.update(mdata,'L2_Round5','17','Myl_Mast')
ct_anno.update(mdata,'L2_Round5','18','Myl_MonoCD14')


ct_anno.update(mdata,'sub','8_1','Myl_DC2')
ct_anno.update(mdata,'sub','10_1','Myl_tDC')
ct_anno.update(mdata,'sub','15_0','Myl_ACY3_DC')
ct_anno.update(mdata,'sub','15_1','Myl_LAMP3_DC')

In [ ]:
m = scjp.markers.marker(mdata,'sub')
marker = m.plot_marker()

In [ ]:
scjp.us(mdata,'anno_final',legend_loc='on data',legend_fontsize=7)

In [ ]:
scjp.model.update_label(mdata,'anno_final',adata,'anno_final','anno_final',replace=True)

In [ ]:
scjp.us(adata,'anno_final',legend_loc='on data',legend_fontsize=7)

In [ ]:
adata.write('./write/%s%s.h5ad'%(version, 'MOG_10X3_anno_hm'))

## B cell subsetting

In [ ]:
B = [x for x in set(sorted(adata.obs['anno_final'])) if 'B_' in x]
B.append('LQ_B')
bdata = scjp.get_subset(adata,adata.obs['anno_final'].isin(B))

In [ ]:
scjp.run_harmony(bdata,'PatientID')

In [ ]:
scjp.us(bdata,'anno_sccaf', legend_loc='on data', legend_fontsize=7)

In [ ]:
sc.tl.leiden(bdata, resolution=2.0, key_added='L2_Round0')

In [ ]:
SCCAF_optimize_all(ad=bdata,min_acc=0.95, start='L2_Round0',prefix='L2',use='pca',basis='umap',c_iter=4)

In [ ]:
# mito, n_genes
scjp.us(bdata,'mito,n_genes')

In [ ]:
scjp.us(bdata,'CD3D,CD4,CD8B,NKG7,KLRB1,IL18R1,FOXP3,CCL5,MS4A1,CD74,PI3,CXCL8,GNG11,LYZ')

In [ ]:
scjp.us(bdata,'L2_Round3',legend_loc='on data',legend_fontsize=6)

In [ ]:
scjp.us(bdata,'leiden',legend_loc='on data',legend_fontsize=6)

In [ ]:
#B_pan
scjp.us(bdata,'MS4A1,CD19,CD74,CD79A')

In [ ]:
#B_naive
scjp.us(bdata,'CD37,IGHD')

In [ ]:
#B_memory
scjp.us(bdata,'CD27,IGHM,CD86')

In [ ]:
#B_plasma
scjp.us(bdata,'IGHG1,JCHAIN,CD38,TNFRSF17')

In [ ]:
#pDC
scjp.us(bdata,'GZMB,IL3RA,CLEC4C,LILRA4,JCHAIN,TCF4,TNFRSF21,JCHAIN,SERPINF1,ITM2C')

In [ ]:
ct_anno = scjp.annotater(bdata,'anno_final',old_label='anno_final')

In [ ]:
ct_anno.update(bdata,'L2_Round3','0','B_naive')
ct_anno.update(bdata,'L2_Round3','1','B_plasma')
ct_anno.update(bdata,'L2_Round3','2','LQ_T')
ct_anno.update(bdata,'L2_Round3','3','B_naive')
ct_anno.update(bdata,'L2_Round3','4','LQ_Mgk')
ct_anno.update(bdata,'L2_Round3','5','B_plasma')
ct_anno.update(bdata,'L2_Round3','6','LQ_T')
ct_anno.update(bdata,'L2_Round3','7','LQ_Myl')
ct_anno.update(bdata,'L2_Round3','8','LQ_Myl')

ct_anno.update(bdata,'L2_Round2','0','B_memory')

In [ ]:
m = scjp.markers.marker(bdata,'L2_Round3')
marker = m.plot_marker()

In [ ]:
scjp.us(bdata,'anno_final',legend_loc='on data',legend_fontsize=7)

In [ ]:
scjp.model.update_label(bdata,'anno_final',adata,'anno_final','anno_final',replace=True)

In [ ]:
scjp.us(adata,'anno_final',legend_loc='on data',legend_fontsize=6)

In [ ]:
adata.write('./write/%s%s.h5ad'%(version, 'MS_public_anno_bef_db_remov'))

In [ ]:
lq = [x for x in sorted(set(adata.obs['anno_final'])) if 'LQ_' in x]

In [ ]:
lq

In [ ]:
bdata = adata.copy()

In [ ]:
adata = adata[~adata.obs['anno_final'].isin(lq)]

In [ ]:
scjp.us(adata,'anno_final',legend_loc='on data',legend_fontsize=6)

In [ ]:
adata

In [ ]:
adata = adata.raw.to_adata()

In [ ]:
scjp.jhk.ad_summary(adata)

In [ ]:
adata.raw = adata

In [ ]:
adata = scjp.remove_geneset(adata,scjp.cc_genes)

In [ ]:
scjp.sc_process(adata,pid = 'fsp')

In [ ]:
scjp.sc_process(adata,pid='ku')

In [ ]:
scjp.us(adata,'PatientID')

In [ ]:
adata.obsm['X_umap_original'] = adata.obsm['X_umap']

## Harmony

In [ ]:
sce.pp.harmony_integrate(adata,'PatientID',adjusted_basis='X_pca')
scjp.sc_process(adata,pid = 'ku')

In [ ]:
scjp.us(adata,'PatientID')

In [ ]:
sc.tl.leiden(adata,resolution=0.7)

In [ ]:
scjp.us(adata,'anno_final',legend_loc='on data',legend_fontsize=6)

In [ ]:
scjp.us(adata,'anno_final')

In [ ]:
scjp.us(adata,'leiden',legend_loc='on data',legend_fontsize=6)

In [ ]:
#Microglia
scjp.us(adata,'TMEM119,CX3CR1,TREM2,GPR34,P2RY12')

In [ ]:
ct_anno = scjp.annotater(adata,'anno_final',old_label='anno_final')

In [ ]:
ct_anno.update(adata,'leiden','9','Myl_Microglia')

In [ ]:
adata.write('./write/%s%s.h5ad'%(version, 'MS_public_anno_db_remov'))